In [37]:
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import pandas as pd
import db_dtypes     

import pandas_gbq

In [38]:
key_path = glob.glob("*.json")[0]   # glob.glob("*.json") json 확장자 다 가져옴
credentials = service_account.Credentials.from_service_account_file(key_path)

In [39]:
dfdf = pd.read_csv('Data/stock_data.csv')

In [23]:
dfdf

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2023-01-02,55500,56100,55200,55500,10031448,558433491400,0.36,005930,삼성전자,KOSPI
1,2023-01-03,55400,56000,54500,55400,13547030,747898872200,-0.18,005930,삼성전자,KOSPI
2,2023-01-04,55700,58000,55600,57800,20188071,1151473733800,4.33,005930,삼성전자,KOSPI
3,2023-01-05,58200,58800,57600,58200,15682826,911973904900,0.69,005930,삼성전자,KOSPI
4,2023-01-06,58300,59400,57900,59000,17334989,1018769177900,1.37,005930,삼성전자,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...
315107,2023-06-16,280,280,212,212,122,32664,-13.11,308700,테크엔,KONEX
315108,2023-06-19,243,243,221,221,51,12371,4.25,308700,테크엔,KONEX
315109,2023-06-20,254,254,222,222,51,12922,0.45,308700,테크엔,KONEX
315110,2023-06-21,255,255,255,255,48,12240,14.86,308700,테크엔,KONEX


In [40]:
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [41]:
# `gcp-study-387610.data_study.iris_sample`
client

In [36]:
sql = f"""
SELECT *
FROM gcp-study-387610.data_study.iris_sample
"""
query_job = client.query(sql)
df = query_job.to_dataframe()
df.head()

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/gcp-study-387610/jobs?prettyPrint=false: Access Denied: Project gcp-study-387610: User does not have bigquery.jobs.create permission in project gcp-study-387610.

Location: None
Job ID: c4cc2f82-353e-425f-bc2c-fd7e71ae13cf



### bigquery create dataset

In [42]:
# table id
# dataset_id = '[prject].[dataset id]'

dataset_id = 'gcp-study-387610.stock_data'

# create dataset
# dataset = bigquery.Dataset(dataset_id)
dataset = bigquery.Dataset(dataset_id)

# dataset region
# dataset.location = ''
dataset.location = 'asia-northeast3'

# createdataset
# dataset = client.create_dateaset(dateaset, timeout= 30)
dataset = client.create_dataset(dataset, timeout = 30)

### create table

In [64]:
# 테이블 ID - '프로젝트 이름.데이터 세트 이름.테이블 이름' 형식으로 입력
# table_id = "[프로젝트 이름].[데이터 세트 이름].[테이블 이름]"
table_id = "gcp-study-387610.stock_data.asdf555"
# 스키마 객체 생성
# 0   날짜         315112 non-null  object 
#  1   시가         315112 non-null  int64  
#  2   고가         315112 non-null  int64  
#  3   저가         315112 non-null  int64  
#  4   종가         315112 non-null  int64  
#  5   거래량        315112 non-null  int64  
#  6   거래대금       315112 non-null  int64  
#  7   등락률        315112 non-null  float64
#  8   ticker     315112 non-null  object 
#  9   corp_name  315112 non-null  object 
#  10  market     315112 non-null  object 
schema = [
bigquery.SchemaField("날짜","STRING"),
bigquery.SchemaField("시가","FLOAT64"),
bigquery.SchemaField("고가","FLOAT64"),
bigquery.SchemaField("저가","FLOAT64"),
bigquery.SchemaField("종가","FLOAT64"),
bigquery.SchemaField("거래량","FLOAT64"),
bigquery.SchemaField("거래대금","FLOAT64"),
bigquery.SchemaField("등락률","FLOAT64"),
bigquery.SchemaField("ticker","STRING"),
bigquery.SchemaField("corp_name","STRING"),
bigquery.SchemaField("market","STRING")

    
]
# 테이블 객체 생성
table = bigquery.Table(table_id, schema=schema)
# 테이블 생성
table = client.create_table(table)

### to_gbq

In [69]:
project_id = 'gcp-study-387610'
table_id = 'stock_data.df1'

In [74]:
dfdf1 =dfdf.head(100)
dfdf1.head(2)
dfdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315112 entries, 0 to 315111
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   날짜         315112 non-null  object 
 1   시가         315112 non-null  int64  
 2   고가         315112 non-null  int64  
 3   저가         315112 non-null  int64  
 4   종가         315112 non-null  int64  
 5   거래량        315112 non-null  int64  
 6   거래대금       315112 non-null  int64  
 7   등락률        315112 non-null  float64
 8   ticker     315112 non-null  object 
 9   corp_name  315112 non-null  object 
 10  market     315112 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 26.4+ MB


In [71]:
pandas_gbq.to_gbq(dfdf1, table_id, 
                  project_id= project_id, credentials = credentials)

100%|██████████| 1/1 [00:00<00:00, 7781.64it/s]


In [53]:
dfdf[dfdf['ticker'] == '260870']

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
70074,2023-01-02,40200,40800,39900,40200,3714,149536600,0.50,260870,SK시그넷,KONEX
70075,2023-01-03,40200,40200,39000,40000,4765,189417500,-0.50,260870,SK시그넷,KONEX
70076,2023-01-04,40000,40000,39250,39800,3233,128409000,-0.50,260870,SK시그넷,KONEX
70077,2023-01-05,40000,40150,39400,39800,2105,83409300,0.00,260870,SK시그넷,KONEX
70078,2023-01-06,39800,40000,39700,39950,5830,232305050,0.38,260870,SK시그넷,KONEX
...,...,...,...,...,...,...,...,...,...,...,...
70186,2023-06-16,54900,55300,54400,55000,18411,1010164000,0.18,260870,SK시그넷,KONEX
70187,2023-06-19,55500,55500,53800,54700,17836,971464300,-0.55,260870,SK시그넷,KONEX
70188,2023-06-20,51400,54700,51400,54700,17064,908204500,0.00,260870,SK시그넷,KONEX
70189,2023-06-21,54800,55400,53800,53900,14666,800155100,-1.46,260870,SK시그넷,KONEX


### create table

In [62]:
dfdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315112 entries, 0 to 315111
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   날짜         315112 non-null  object 
 1   시가         315112 non-null  int64  
 2   고가         315112 non-null  int64  
 3   저가         315112 non-null  int64  
 4   종가         315112 non-null  int64  
 5   거래량        315112 non-null  int64  
 6   거래대금       315112 non-null  int64  
 7   등락률        315112 non-null  float64
 8   ticker     315112 non-null  object 
 9   corp_name  315112 non-null  object 
 10  market     315112 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 26.4+ MB


In [58]:
dfdf

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,ticker,corp_name,market
0,2023-01-02,55500,56100,55200,55500,10031448,558433491400,0.36,005930,삼성전자,KOSPI
1,2023-01-03,55400,56000,54500,55400,13547030,747898872200,-0.18,005930,삼성전자,KOSPI
2,2023-01-04,55700,58000,55600,57800,20188071,1151473733800,4.33,005930,삼성전자,KOSPI
3,2023-01-05,58200,58800,57600,58200,15682826,911973904900,0.69,005930,삼성전자,KOSPI
4,2023-01-06,58300,59400,57900,59000,17334989,1018769177900,1.37,005930,삼성전자,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...
315107,2023-06-16,280,280,212,212,122,32664,-13.11,308700,테크엔,KONEX
315108,2023-06-19,243,243,221,221,51,12371,4.25,308700,테크엔,KONEX
315109,2023-06-20,254,254,222,222,51,12922,0.45,308700,테크엔,KONEX
315110,2023-06-21,255,255,255,255,48,12240,14.86,308700,테크엔,KONEX


In [ ]:
sql = f"""
SELECT *
FROM gcp-study-387610.data_study.iris_sample
"""
query_job = client.query(sql)

In [ ]:
df = query_job.to_dataframe()
df.head()

In [15]:
dfdf1 =  dfdf[['날짜','종가','ticker','corp_name','market']]

In [18]:
dfdf_22 = dfdf1[dfdf1['market'] == 'KOSDAQ']

In [19]:
dfdf_22.to_csv('Data/kosdaq_data.csv')